# Memoization with dictionaries

The previous notebook defined a recursive function for computing Fibonacci numbers.

In [1]:
def fibonacci(n):
    """Recursive implementation of Fibonacci series.
    
    Only defined for n >= 1."""
    # base case for the first two numbers
    if n == 1 or n == 2:
        return 1
    # recursion step
    else:
        return fibonacci(n-1) + fibonacci(n-2)

Even `fibonacci(50)` takes forever to compute this way.
The culprit is that the recursion causes the same values to be computed over and over again.

### Lightning-fast Fibonacci numbers with memoization

Memoization means that we sacrifice some memory to store the results of intermediate computations.
In Python, the standard tool for this is a dictionary, but depending on the problem other data structures might also work.
All we have to do is to modify our recursive function so that it stores the value for each Fibonacci number in a dictionary.
The value of a Fibonacci number will be computed only if it isn't already in the dictionary.

In [1]:
# inititalize memo,
memo = {}
# and add values for the first two Fibonacci numbers
memo[1], memo[2] = 1, 1
    
def fibonacci_memo(n):
    """Recursive implementation of Fibonacci series, with memoization.
    
    Only defined for n >= 1."""
    # new base case: if value is in memo, get it from there
    if memo.get(n):
        return memo[n]
    # otherwise, compute recursively but store the result in memo
    else:
        memo[n] = fibonacci_memo(n-2) + fibonacci_memo(n-1)
        return memo[n]

By adding values to `memo` the first time they are computed, this function works much faster.

In [2]:
# tell Python to allow plenty of recursion;
# see the end of this notebook for details on sys.setrecursionlimit
import sys
sys.setrecursionlimit(100000)

# let's see how fast this is now!
for n in [50, 100, 1000, 10000]:
    print(f"The {n}th Fibonacci number is:")
    print(fibonacci_memo(n))

The 50th Fibonacci number is:
12586269025
The 100th Fibonacci number is:
354224848179261915075
The 1000th Fibonacci number is:
43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875
The 10000th Fibonacci number is:
336447648764317832666216120051075433103021484606800639065647699746800814421666623681555955136337340255820653326808361593737347904838652682630408924630564318873545443695598274916066020998841839338646527313000888302692356736131351175792974378544137521305205043477016022647583189065278908551543661595829872796829875106312005754287834532155151038708182989697916131278562650331954871402142875326981879620469360978799003509623022910263681314931952756302278376284415403605844025721143349611800230912082870460889239623288354615057765832712525460935911282039252853934346209042452489294039017062338889910858410651831733604374707379085526317

We've gone from an entire coffee break for the 50th Fibonacci number to a split second for the 50,000th.
Now that's one giant speed-up!
And that's why memoization is your best friend.

Let's look once again at some diagrams to see how exactly memoization achieves this speed-up.
First the diagram for `fibonacci_memo(5)`.

```
fibonacci_memo(5)
     |
fibonacci_memo(3) + fibonacci_memo(4) (--> add 5: 5 to memo)
     |                   |
     |              fibonacci_memo(3) + fibonacci_memo(2) (--> add 4: 3 to memo)
     |                   |                  |
     |                   2 from memo        1 from memo
     |
     |
     |
     |
     |
     |
     |
fibonacci_memo(2) + fibonacci_memo(1) (--> add 3: 2 to memo)
     |                   |
     1 from memo         1 from memo
```

Contrast this diagram with the one we saw earlier for `fibonacci(5)`, without any memoization.
The crucial difference is that `fibonacci_memo(3)` is only computed once.
After that, its value is immediately retrieved from the dictionary.
This makes everything much faster.
The diagram for `fibonacci_memo(7)`, for instance, isn't much larger than that for `fibonacci_memo(5)`.

```
fibonacci_memo(7)
     |
fibonacci_memo(5) + fibonacci_memo(6)
     |                   |
     |              fibonacci_memo(4) + fibonacci_memo(5)
     |                   |                   |
     |                   3 from memo         5 from memo   
     |
fibonacci_memo(3) + fibonacci_memo(4) (--> add 5: 5 to memo)
     |                   |
     |              fibonacci_memo(3) + fibonacci_memo(2) (--> add 4: 3 to memo)
     |                   |                  |
     |                   2 from memo        1 from memo
     |
     |
     |
     |
     |
     |
     |
fibonacci_memo(2) + fibonacci_memo(1) (--> add 3: 2 to memo)
     |                   |
     1 from memo         1 from memo
```

Without memoization, `fibonacci(7)` would be huge and wouldn't fit on the screen (try drawing its diagram - you'll need a big piece of paper).
With memoziation, the step from `fibonacci(5)` to `fibonacci(7)` only adds two additional summing steps:

1. `3 + 5` for `fibonacci_memo(6)`, and
1. `5 + 8` for `fibonacci_memo(7)`.

So the step from computing Fibonacci number `n` to computing Fibonacci number `n+1` is just one more addition step.
That's cheap because addition is fairly cheap.
Now the difference between computing the 10th Fibonacci number and the 50th is no more challenging that adding 10 numbers versus adding 50.
In other words, it's pretty trivial as far as Python is concerned.

And this is why memoization is an essential component of every programmer's toolkit.
If you want to see just how badly the recursive function scales without memoization, you can use the `fib_waste` function below to see how often each value would be recomputed.

In [3]:
def fib_waste(n):
    memo = {}
    memo[1], memo[2] = 1, 1
    record = {}
    
    record[n] = 1
    if n > 2:
        for m in range(1, n):
            record[n - m] = fibonacci_memo(m + 1)
        record[1] = fibonacci_memo(n - 2)
    return record

In [4]:
# compute the n-th Fibonacci number; change as you wish
n = 50
# you can ignore all the code below

# tabulate allows us to print tables
from tabulate import tabulate
# make numbers more readable with commas
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')


def comma_numbers(n):
    return locale.format_string("%d", n, grouping=True)

record = fib_waste(n)
table = {key: comma_numbers(val) for key, val in fib_waste(n).items()}
print(tabulate(table.items(), headers=["Fibonacci number", "times computed"], stralign="right"))
print("Total number of computations:", comma_numbers(sum(record.values())))

  Fibonacci number    times computed
------------------  ----------------
                50                 1
                49                 1
                48                 2
                47                 3
                46                 5
                45                 8
                44                13
                43                21
                42                34
                41                55
                40                89
                39               144
                38               233
                37               377
                36               610
                35               987
                34             1,597
                33             2,584
                32             4,181
                31             6,765
                30            10,946
                29            17,711
                28            28,657
                27            46,368
                26            75,025
 

## Avoiding recursion altogether

Here's a dirty secret: we don't actually need recursive functions to compute Fibonacci numbers.
We used a recursive function because Fibonacci numbers are an excellent example of how powerful a tool memoization can be.
In practice, though, it makes more sense to compute the Fibonacci series with a list.

In [5]:
def fibonacci_nonrecursive(n):
    fib = [1,1]
    for m in range(3, n+1):
        fib.append(fib[-2] + fib[-1])
    return fib[-1]

This is pretty much how one would naively compute a Fibonacci number.
You start with `1` and `1`, and then you keep adding sums of the previous two numbers until you've reached the point where you want to be.
A simple but effective solution.
Notice how we implicitly use memoization by storing each Fibonacci number in the list `fib`.

This implementation also computes exactly the same values as `fibonacci_memo`, so it works as desired even though it doesn't use recursion at all.

In [6]:
for n in ([1,2,3,4,5,10,50,100]):
    if fibonacci_memo(n) != fibonacci_nonrecursive(n):
        print("Discrepancy found")

It is also very fast.
Just like `fibonacci_memo`, the `fibonacci_nonrecursive` implementation does not choke on large Fibonacci numbers.

In [8]:
for n in [50, 100, 1000, 10000]:
    print(f"The {n}th Fibonacci number is:")
    print(fibonacci_nonrecursive(n))

The 50th Fibonacci number is:
12586269025
The 100th Fibonacci number is:
354224848179261915075
The 1000th Fibonacci number is:
43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875
The 10000th Fibonacci number is:
336447648764317832666216120051075433103021484606800639065647699746800814421666623681555955136337340255820653326808361593737347904838652682630408924630564318873545443695598274916066020998841839338646527313000888302692356736131351175792974378544137521305205043477016022647583189065278908551543661595829872796829875106312005754287834532155151038708182989697916131278562650331954871402142875326981879620469360978799003509623022910263681314931952756302278376284415403605844025721143349611800230912082870460889239623288354615057765832712525460935911282039252853934346209042452489294039017062338889910858410651831733604374707379085526317

When timing them, they both take only a few microseconds on very large numbers.

In [ ]:
% time
fibonacci_nonrecursive(10000)
# hide output of function
;

In [ ]:
% time
fibonacci_memo(10000)
# hide output of function
;

And in fact, `fibonacci_nonrecursive` is better for very large numbers.
For instance, there's no problem with computing the 100,000th Fibonacci number with the list approach, but the memoization approach may actually cause the Python kernel to crash.

In [ ]:
# this should work fine
fibonacci_nonrecursive(100000)

In [ ]:
# this might crash
fibonacci_memo(100000)

And even if it doesn't crash, the recursive function may run into problems with Python's *recursion depth*.
This parameter controls how many nested recursion steps are allowed.
In `fibonacci_memo`, a larger Fibonacci number takes more recursion steps to compute.
So large Fibonacci numbers cannot be computed without hitting the recursion depth ceiling.
We can see this by manually lowering the stack depth with `sys.setrecursionlimit`.

In [21]:
import sys
sys.setrecursionlimit(50)
print(fibonacci_memo(50000))          # gives RecursionError

RecursionError: maximum recursion depth exceeded while calling a Python object

For all intents and purpose, the recursive function approach simply doesn't work well in Python, and memoization is just a band-aid to make it useable.
But this is mostly just because Python's handling of recursive functions is subpar compared to other languages.
In the programming language [Haskell](https://en.wikipedia.org/wiki/Haskell_(programming_language)), for instance, recursive functions are an essential part of every program, and they work fast and reliably.
A good programmer should know how to translate a `for`-loop or `while`-loop into a recursive function.
But they should also know when this is a smart thing to do.
And for Python, the answer is pretty much "never".

This does ont mean, though, that memoization is useless.
Even with non-recursive functions, memoization can be essential for reaching reasonable speed.
So even though our motivation for memoization was recursive functions, the problems with recursive functions in Python do not mean that memoization has no use in Python.
Quite the opposite.
Now that you're hopefully more comofortable with memoization, we can finally leave mathematics and recursive functions behind us and look at a language-related problem that involves dynamic program.
Yes, it's finally time to look at how the Levenshtein distance is implemented.

## Bullet point summary

- Memoization uses a data structure (e.g. a dictionary) to store intermediate values.
- Memoization can tremendously cut down computation time (from hours to seconds).
- Memoization often involves a **time-space trade-off**: it decreases overall coputation time, but your program might take up more space in memory.
- Recursive functions should be avoided in Python: a normal loop will be faster, doesn't require specialized memoization techniques, and isn't bounded by Python's recursion depth ceiling.
- Nonetheless, recursion is an essential programming technique, and many other languages rely heavily on it.